<a href="https://colab.research.google.com/github/navidadkhah/RAG/blob/main/Creating_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chromadb

In [ ]:
import chromadb
import pandas as pd
import numpy as np

In [ ]:
chroma_client = chromadb.PersistentClient(path="/content/drive/MyDrive/RAG/new_vector")
collection = chroma_client.get_or_create_collection(name="my_collection")
print(collection.count())
collection.peek()

0


{'ids': [],
 'embeddings': array([], dtype=float64),
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents', 'embeddings'],
 'data': None,
 'metadatas': []}

In [ ]:
path = "/content/drive/MyDrive/RAG/movies_&_series.csv"
df = pd.read_csv(path)
df.head()

,Unnamed: 0,Name,overview,genres,language,popularity
0,0,Havoc,When a drug heist swerves lethally out of cont...,"Action, Crime, Thriller",en,575.1039
1,1,A Working Man,Levon Cade left behind a decorated military ca...,"Action, Crime, Thriller",en,548.9128
2,2,新幹線大爆破,When panic erupts on a Tokyo-bound bullet trai...,"Action, Thriller, Crime, Drama",ja,276.1329
3,3,In the Lost Lands,A queen sends the powerful and feared sorceres...,"Action, Fantasy, Adventure",en,275.3451
4,4,ज्वेल थीफ: द हीस्ट बिगिन्स,"In this high-octane battle of wits and wills, ...","Action, Thriller",hi,273.4631


In [ ]:
df['combined_text'] = (
    "Title: " + df['Name'].astype(str) + ". " +
    "Overview: " + df['overview'].astype(str) + ". " +
    "Genres: " + df['genres'].astype(str) + ". " +
    "Language: " + df['language'].astype(str) + ". " +
    "Popularity: " + df['popularity'].astype(str)
)
print(df.shape)
df.head()

(180442, 7)


,Unnamed: 0,Name,overview,genres,language,popularity,combined_text
0,0,Havoc,When a drug heist swerves lethally out of cont...,"Action, Crime, Thriller",en,575.1039,Title: Havoc. Overview: When a drug heist swer...
1,1,A Working Man,Levon Cade left behind a decorated military ca...,"Action, Crime, Thriller",en,548.9128,Title: A Working Man. Overview: Levon Cade lef...
2,2,新幹線大爆破,When panic erupts on a Tokyo-bound bullet trai...,"Action, Thriller, Crime, Drama",ja,276.1329,Title: 新幹線大爆破. Overview: When panic erupts on ...
3,3,In the Lost Lands,A queen sends the powerful and feared sorceres...,"Action, Fantasy, Adventure",en,275.3451,Title: In the Lost Lands. Overview: A queen se...
4,4,ज्वेल थीफ: द हीस्ट बिगिन्स,"In this high-octane battle of wits and wills, ...","Action, Thriller",hi,273.4631,Title: ज्वेल थीफ: द हीस्ट बिगिन्स. Overview: I...


In [ ]:
embeddings = np.load('/content/drive/MyDrive/RAG/embeddings.npy', allow_pickle=True)
print(embeddings.shape)

ValueError: cannot reshape array of size 2097120 into shape (180442,384)

In [ ]:
starter = 0
index = 100

while starter < 12000:
  vectors = df["combined_text"][starter:starter+index].to_list()
  ids = [f"id{i}" for i in range(starter, starter+index)]
  collection.add(
      documents=vectors,
      ids=ids
      )

  starter += index
  print(collection.count())

print(f"----- {collection.count()} of rows are added. -----")


100


In [ ]:
print(collection.count())
collection.peek()

NameError: name 'collection' is not defined

In [ ]:
collection.peek()

{'ids': ['id1', 'id2', 'id3', 'id4', 'id5'],
 'embeddings': array([[ 0.03738717,  0.04748575, -0.05010892, ...,  0.03359606,
          0.02930197, -0.01267304],
        [-0.13455388, -0.02684624, -0.10108222, ..., -0.03512937,
         -0.0144708 ,  0.00107189],
        [-0.00158789, -0.01445741,  0.03929684, ...,  0.00343756,
         -0.0712861 ,  0.03635733],
        [-0.00597597,  0.01764174, -0.01465255, ..., -0.02730393,
         -0.05556757, -0.02783172],
        [-0.03257563,  0.07209565, -0.06889339, ..., -0.0201755 ,
         -0.06639002, -0.0510019 ]]),
 'documents': ["Title: Havoc. Overview: When a drug heist swerves lethally out of control, a jaded cop fights his way through a corrupt city's criminal underworld to save a politician's son.. Genres: Action, Crime, Thriller. Language: en. Popularity: 575.1039",
  "Title: A Working Man. Overview: Levon Cade left behind a decorated military career in the black ops to live a simple life working construction. But when his boss's 

In [ ]:
results = collection.query(
    query_texts=["I want a movie in a lost world"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
results

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 77.4MiB/s]


{'ids': [[]],
 'embeddings': None,
 'documents': [[]],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[]],
 'distances': [[]]}

In [ ]:
user_input = input("write your command: ")

retrieved_movies = results["documents"]

context = "\n\n".join([f"Movie {i+1}: {doc}" for i, doc in enumerate(retrieved_movies)])

prompt = f"""
You are an expert film curator and critic with a deep understanding of cinema, narrative themes, and genres.

A user has requested a movie recommendation for the theme: "{user_input}" Below are two detailed movie summaries retrieved from a specialized movie database:

{context}

Please do the following:
1. Identify which of these films most compellingly fits the user's request, providing a thoughtful comparison between the options.
2. If relevant, mention what makes the second option less ideal for this particular request.

Your recommendation should sound authoritative, enthusiastic, and helpful—like a specialist at a film festival or a trusted movie guide.
"""

prompt